<a href="https://colab.research.google.com/github/r-hugoalves/Projeto_Completo_Analise_Censo_2010/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análise do Censo de 2010

In [ ]:
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#Importação do arquivo XML
with open(file='./estados_bruto.xml', mode='r') as arquivo_xml:
  dados_xml = arquivo_xml.read()

dados_estados = BeautifulSoup(dados_xml, 'xml')

#Extraindo a tag ID
tags_id = dados_estados.find_all('ID')
#Extraindo a tag NOME
tags_nome = dados_estados.find_all('NOME')
#Extraindo a tag IDCAPITAL
tags_idcapital = dados_estados.find_all('IDCAPITAL')
#Extraindo a tag SIGLA
tags_sigla = dados_estados.find_all('SIGLA')
#Extraindo a tag REGIAO
tags_regiao = dados_estados.find_all('REGIAO')

In [ ]:
#Vamos agora tratar as listas e escrever as informações no arquivo .CSV

with open(file='estados-limpo.csv', mode='w', encoding='utf8') as arquivo:
  escritor_csv = csv.writer(arquivo, delimiter=';')
  escritor_csv.writerow(['ID_ESTADO', 'UF', 'CAPITAL', 'SIGLA', 'REGIAO'])

  for i in range(0,len(tags_id)):
    escritor_csv.writerow([
        tags_id[i].get_text(),
        tags_nome[i].get_text(),
        tags_idcapital[i].get_text(),
        tags_sigla[i].get_text(),
        tags_regiao[i].get_text()
    ])

In [ ]:
#Agora, vamos criar um dataframe com o arquivo CSV selecionando apenas as colunas de interesse:
  #'UF', 'nome', 'PIB', 'Pop_est_2009', 'PIB_percapita'

cidades_df = pd.read_csv('cidades-bruto.csv', sep=',')
dados_cidades_df = cidades_df[['UF', 'nome', 'PIB', 'Pop_est_2009', 'PIB_percapita']]

#Para melhor visualização, vamos alterar o nome da coluna 'nome' para 'Cidade'

dados_cidades_df = dados_cidades_df.rename(columns={'nome': 'Cidade'})

In [ ]:
#Transformando o arquivo estados-limpo.csv em um datraframe para fazer a combinação

estados_df = pd.read_csv('estados-limpo.csv', sep=';')

#Aqui tem um problema que pode gerar confusão na hora de combinar
  #Os estados estão sem acentos, e no 'dados_cidades_df' eles possuem acentos. Vamos tratar isso
  #A única exceção é Espírito Santo que está sem acento nas duas tabelas

estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'AMAPÁ' if acento == 'AMAPA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'CEARÁ' if acento == 'CEARA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'GOIÁS' if acento == 'GOIAS' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'MARANHÃO' if acento == 'MARANHAO' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'PARÁ' if acento == 'PARA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'PARANÁ' if acento == 'PARANA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'PARAÍBA' if acento == 'PARAIBA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'PIAUÍ' if acento == 'PIAUI' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'RONDÔNIA' if acento == 'RONDONIA' else acento)
estados_df['UF'] = estados_df['UF'].apply(lambda acento: 'SÃO PAULO' if acento == 'SAO PAULO' else acento)

In [ ]:
#Junção dos dataframes

brasil_df_bruto= pd.merge(left=estados_df, right=dados_cidades_df, on='UF', how='outer')

#Existe uma cidade em específico, de PIB: 86340.6015625 que tem o nome dela como NaN. Vamos tratar:

brasil_df = brasil_df_bruto.fillna('')

#Vamos converter o brasil_df para um arquivo csv para que possamos fazer as análises no PowerBI

brasil_df.to_csv('analise_censo_2010.csv', index=False, sep=';', encoding='utf8')